In [32]:
# Data Science
import re
import csv
import json
import itertools
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
from pandasgui import show
import matplotlib.pyplot as plt

# General
import os
import sys
import time
import math
import time
import random
from datetime import date
import warnings
current_date = date.today()
warnings.filterwarnings("ignore")

# ML
from sklearn.model_selection import train_test_split

# NLP
import torch
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from gensim.parsing.preprocessing import remove_stopwords
from simpletransformers.language_representation import RepresentationModel
from transformers import BertTokenizer
from gensim.models import Word2Vec 
import spacy

spacy.prefer_gpu()
%load_ext autotime
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 3500000

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 906 ms


In [34]:
cuda_available = torch.cuda.is_available()
if cuda_available:
    print(">>> CUDA is available")
else:
    print(">>> CUDA is not available, we use CPU instead.")

>>> CUDA is not available, we use CPU instead.
time: 0 ns


## Train-Test-Split

In [33]:
train_df = pd.read_csv("input_optimized.csv") 

train_df = train_df[['Unnamed: 0', 'patient_id', 'sequence','original', 'annotator_label']]

time: 266 ms


In [36]:
X = train_df["sequence"]
y = train_df["annotator_label"]

y_label = y.to_numpy()
X_train, X_test_valid, y_train, y_test_valid = train_test_split(X,y,random_state=0,test_size=0.1, stratify=y_label)

y_test_valid_label = y_test_valid.to_numpy()
X_valid, X_test, y_valid, y_test = train_test_split(X_test_valid, y_test_valid, random_state=0, test_size=0.25, stratify=y_test_valid_label)

time: 31 ms


In [37]:
manual_review = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\Modeling\test_and_validation.csv")

time: 46 ms


In [38]:
X_2 = manual_review["sequence"]
y_2 = manual_review["annotator_label"]

y_label_2 = y_2.to_numpy()
X_train_2, X_test_valid_2, y_train_2, y_test_valid_2 = train_test_split(X_2,y_2,random_state=0,test_size=0.6, stratify=y_label_2)

y_test_valid_label_2 = y_test_valid_2.to_numpy()
X_valid_2, X_test_2, y_valid_2, y_test_2 = train_test_split(X_test_valid_2, y_test_valid_2, random_state=0, test_size=(0.25/0.6), stratify=y_test_valid_label_2)

time: 15 ms


In [39]:
X_train = X_train.append(X_train_2)
y_train = y_train.append(y_train_2)

X_test = X_test.append(X_test_2)
y_test = y_test.append(y_test_2)

X_valid = X_valid.append(X_valid_2)
y_valid = y_valid.append(y_valid_2)

time: 16 ms


In [40]:
X_train = X_train.append(X_valid)
y_train = y_train.append(y_valid)

time: 0 ns


In [44]:
manual_review = manual_review[['Unnamed: 0', 'patient_id', 'sequence','original', 'annotator_label']]

time: 0 ns


In [45]:
df = pd.concat([train_df, manual_review])

time: 0 ns


In [55]:
df['split'] = ['not_set']*df.shape[0]

time: 0 ns


In [82]:
X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)
df = df.reset_index(drop = True)

time: 0 ns


In [70]:
row = df[df["sequence"] == X_train[0]]
df.loc[df["sequence"] == X_train[i] ["split"]] = "train"

TypeError: string indices must be integers

time: 32 ms


In [76]:
df["split"].value_counts()

train      8302
not_set     354
Name: split, dtype: int64

time: 16 ms


In [83]:
for i in tqdm(range(len(X_train))):
    df.loc[df["sequence"] == X_train[i], ["split"]] = "train"

100%|█████████████████████████████████████████████████████████████████████████████| 8302/8302 [00:15<00:00, 534.21it/s]

time: 15.5 s


In [88]:
for i in tqdm(range(len(X_test))):
    df.loc[df["sequence"] == X_test[i], ["split"]] = "test"

100%|███████████████████████████████████████████████████████████████████████████████| 354/354 [00:00<00:00, 556.52it/s]

time: 641 ms


In [89]:
df

,Unnamed: 0,patient_id,sequence,original,annotator_label,data_type,split
0,0,Z15564314,s other free text-see phs viewer social histor...,s other free text-see phs viewer Social Histor...,1,not_set,train
1,1,Z10171706,------- fusion: no sleep disturbance: no socia...,------- fusion: No Sleep disturbance: No Socia...,1,not_set,train
2,2,Z8935348,ain spasm). 30 tablet 0 unknown (outside pharm...,ain/spasm). 30 tablet 0 Unknown (outside pharm...,1,not_set,train
3,3,Z12212893,------- on 112mcg dose. maria will check dose ...,------- on 112mcg dose. Maria will check dos...,2,not_set,train
4,4,Z9598376,------- (vibramycin) 100 mg capsule take 1 cap...,------- (VIBRAMYCIN) 100 MG capsule Take 1 ca...,1,not_set,train
...,...,...,...,...,...,...,...
8651,8594,Z9536296,ecreased contraversive pushing. ot: pt tolerat...,ecreased contraversive pushing. OT: Pt tolerat...,0,not_set,test
8652,8605,Z9857663,------- not tolerate cpap following with sleep...,------- not tolerate CPAP/ Following with Sle...,0,not_set,test
8653,8606,Z12110231,---------------------------------------follow-...,---------------------------------------Follow-...,0,not_set,train
8654,8609,Z8822410,le. 2. no acute fracture or dislocation of the...,le. 2. No acute fracture or dislocation of the...,0,not_set,train


time: 31 ms


In [28]:
seq_len = 1024
num_samples = len(df)

# initialize empty zero arrays
Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))

# check shape
Xids.shape

(8656, 1024)

time: 0 ns


In [31]:
# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', do_lower_case=True)

for i, phrase in enumerate(df['sequence']):
    tokens = tokenizer.encode_plus(phrase, max_length=seq_len, truncation=True,
                                   padding='max_length', add_special_tokens=True,
                                   return_tensors='pt')
    # assign tokenized outputs to respective rows in numpy arrays
    Xids[i, :] = tokens['input_ids']
    Xmask[i, :] = tokens['attention_mask']

time: 40.4 s


In [30]:
# first extract sentiment column
arr = df['label'].values

# we then initialize the zero array
labels = np.zeros((num_samples, arr.max()+1))

# set relevant index for each row to 1 (one-hot encode)
labels[np.arange(num_samples), arr] = 1

time: 16 ms


In [25]:
def seq_embedding_generation(text):
    seq_embed_list = []
    seq_vectors = model.encode_sentences(text, combine_strategy="mean")
    seq_vectors = np.mean(seq_vectors, axis = 0)
    seq_embed_list.append(seq_vectors.tolist())
    #print("Generated note vector shape as {}".format(seq_vectors.shape))
    return seq_embed_list

time: 0 ns


In [26]:
df["note_embeddings"] = df["sequence"].apply(lambda x: seq_embedding_generation(x))

KeyboardInterrupt: 

time: 18min 14s


In [27]:
df

,Unnamed: 0,patient_id,sequence,original,label
0,0,Z15564314,s other free text-see phs viewer social histor...,s other free text-see phs viewer Social Histor...,1
1,1,Z10171706,------- fusion: no sleep disturbance: no socia...,------- fusion: No Sleep disturbance: No Socia...,1
2,2,Z8935348,ain spasm). 30 tablet 0 unknown (outside pharm...,ain/spasm). 30 tablet 0 Unknown (outside pharm...,1
3,3,Z12212893,------- on 112mcg dose. maria will check dose ...,------- on 112mcg dose. Maria will check dos...,2
4,4,Z9598376,------- (vibramycin) 100 mg capsule take 1 cap...,------- (VIBRAMYCIN) 100 MG capsule Take 1 ca...,1
...,...,...,...,...,...
8651,8651,Z6595984,------- gnitive concerns. interim history pati...,------- gnitive concerns. Interim History Pati...,2
8652,8652,Z10123442,"------- rm and well perfused, no evidence of e...","------- rm and well perfused, no evidence of ...",0
8653,8653,Z6694483,: 77.7 kg (171 lb 4.8 oz) spo2: 98% 99% 98% cu...,: 77.7 kg (171 lb 4.8 oz) SpO2: 98% 99% 98% Cu...,0
8654,8654,Z9195765,mr. donovan lives with his wife with whom he h...,Mr. Donovan lives with his wife with whom he ...,1


time: 32 ms
